In [5]:
import pandas as pd

combined_data = pd.read_csv('combined_data.csv').set_index('video_id')
label = combined_data['label']
features = combined_data.drop(['label'], axis=1)

In [25]:
# import random
# import numpy as np
# import pandas as pd
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# 
# # Initialize the regression model
# model = LogisticRegression(random_state=42, penalty='l1', solver='liblinear')
# 
# # 1. Initial Population
# def initialise_population(size, n_feat):
#     population = []
#     for _ in range(size):
#         chromosome = np.ones(n_feat, dtype=bool)
#         chromosome[:int(0.3 * n_feat)] = False
#         np.random.shuffle(chromosome)
#         population.append(chromosome)
#     return population
# 
# # 2. Fitness Function
# def fitness_score(population, X_train, X_test, y_train, y_test):
#     scores = []
#     for chromosome in population:
#         X_train_filtered = X_train[:, chromosome]
#         X_test_filtered = X_test[:, chromosome]
#         model.fit(X_train_filtered, y_train)
#         predictions = model.predict(X_test_filtered)
#         mse = mean_squared_error(y_test, predictions)
#         # Use negative MSE because we want to maximize fitness (minimize MSE)
#         scores.append(-mse)
#     scores, population = np.array(scores), np.array(population)
#     inds = np.argsort(scores)[::-1]  # Sort so that individuals with higher scores (lower MSE) come first
#     return list(scores[inds]), list(population[inds])
# 
# # 3. Selection
# def selection(pop_after_fit, n_parents):
#     return pop_after_fit[:n_parents]
# 
# # 4. Crossover
# def crossover(pop_after_sel):
#     population_nextgen = []
#     for i in range(0, len(pop_after_sel)-1, 2):
#         child1, child2 = pop_after_sel[i].copy(), pop_after_sel[i+1].copy()
#         crossover_point = np.random.randint(1, len(child1)-1)
#         child1[crossover_point:], child2[crossover_point:] = child2[crossover_point:], child1[crossover_point:]
#         population_nextgen.extend([child1, child2])
#     return population_nextgen
# 
# # 5. Mutation
# def mutation(pop_after_cross, mutation_rate):
#     population_nextgen = []
#     for chromosome in pop_after_cross:
#         chromosome = np.array([not gene if random.random() < mutation_rate else gene for gene in chromosome])
#         population_nextgen.append(chromosome)
#     return population_nextgen
# 
# # Generations
# def generations(size, n_feat, n_parents, mutation_rate, n_gen, X_train, X_test, y_train, y_test):
#     best_chromo = []
#     best_score = []
#     population_nextgen = initialise_population(size, n_feat)
#     for _ in range(n_gen):
#         scores, pop_after_fit = fitness_score(population_nextgen, X_train, X_test, y_train, y_test)
#         pop_after_sel = selection(pop_after_fit, n_parents)
#         pop_after_cross = crossover(pop_after_sel)
#         population_nextgen = mutation(pop_after_cross, mutation_rate)
#         best_chromo.append(pop_after_fit[0])
#         best_score.append(scores[0])
#     return best_chromo, best_score
# size = 50  # Population size
# n_feat = x_train.shape[1]  # Number of features
# n_parents = 25  # Number of parents for crossover
# mutation_rate = 0.01  # Mutation rate
# n_gen = 100  # Number of generations
# 
# # Record the start time
# start_time = time.time()
# best_chromo, best_score = generations(size, n_feat, n_parents, mutation_rate, n_gen, x_train, x_test, y_train, y_test)
# # Calculate the duration
# end_time = time.time()
# duration = end_time - start_time

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
"""
X,Y - Training Data: Features and Label
x_test, y_test - Testing Data: Features and Label
"""
X, x_test, y, y_test = train_test_split(features.values, label.values, test_size=0.2, random_state=0)

In [ ]:
# https://deap.readthedocs.io/en/master/index.html
# https://deap.readthedocs.io/en/master/tutorials/basic/part1.html


In [3]:
%pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 1.4 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from deap import base, creator, tools, algorithms
import numpy as np
from sklearn.model_selection import train_test_split

# Define problem to DEAP (we aim to minimize RMSE, hence weights=(-1.0,))
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 0, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, n=X.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the fitness function for regression
def evalModel(individual):
    X_selected = X[:, [i for i, bit in enumerate(individual) if bit == 1]]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    if X_train.shape[1] == 0:  # Prevent training with 0 features
        return (float("inf"),)  # Return infinite RMSE as a penalty

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, predictions))  # Calculate RMSE
    return (rmse,)

toolbox.register("evaluate", evalModel)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Parameters for GA
population_size = 50
crossover_probability = 0.5
mutation_probability = 0.2
number_of_generations = 40

pop = toolbox.population(n=population_size)

# Run the Genetic Algorithm
result = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability,
                             mutpb=mutation_probability, ngen=number_of_generations,
                             verbose=True)


/Users/diogo.pessoa/code/video-trends/venv/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	30    
2  	36    
3  	31    
4  	31    
5  	31    
6  	30    
7  	21    
8  	26    
9  	28    
10 	27    


In [ ]:
print("Best Score:", best_score[-1])
# mse = mean_squared_error(y_test, gbr_pred)
# r_two_score = r2_score(y_test, gbr_pred)
# ex_variance_score = explained_variance_score(y_test, gbr_pred)

print(f"GA algorithm  took {duration:.2f} seconds.")
# print(f'Mean Squared Error on Test Set: {mse}')
# print(f'r2 score: {r_two_score}')
# print(f'explained variance score: {ex_variance_score}')

In [ ]:

x_train_genetic = x_train[:, best_chromo[-1]]
x_test_genetic = x_test[:, best_chromo[-1]]
n_estimators=140
# Training classifiers
grad_boosting_regressor.fit(x_train_genetic, y_train)
ada_boosting_regressor.fit(x_train_genetic, y_train)
bagging_regressor.fit(x_train_genetic, y_train)
random_forest_regressor.fit(x_train_genetic, y_train)
linear_regr.fit(x_train_genetic, y_train)
voting_reg.fit(x_train_genetic, y_train) 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

print('mean squared error', 'r2 score', 'explained variance score')
print('GradientBoostingRegressor',mean_squared_error(y_test, gdb_prediction),r2_score(y_test, gdb_prediction), explained_variance_score(y_test, gdb_prediction))
print('RandomForestRegressor',mean_squared_error(y_test, rf_prediction),r2_score(y_test, rf_prediction), explained_variance_score(y_test, rf_prediction))
print('LinearRegression',mean_squared_error(y_test, lg_prediction),r2_score(y_test, lg_prediction), explained_variance_score(y_test, lg_prediction))
print('AdaBoostRegressor',mean_squared_error(y_test, ada_b_prediction),r2_score(y_test, ada_b_prediction), explained_variance_score(y_test, ada_b_prediction))
print('BaggingRegressor',mean_squared_error(y_test, bagging_r_prediction),r2_score(y_test, bagging_r_prediction), explained_variance_score(y_test, bagging_r_prediction))
print('VotingRegressor',mean_squared_error(y_test, voting_reg_prediction),r2_score(y_test, voting_reg_prediction), explained_variance_score(y_test, voting_reg_prediction))